**PRELIMINARY NOTEBOOK SETUP**

In [ ]:
# load necessary imports
%pip install google-cloud-bigquery-storage
from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

# Load credentials (adjust path as needed)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/callummagarian/Desktop/Machine-Learning-Final/credentials/filepath.json"
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


False

In [ ]:
# define BigQuery client
client = bigquery.Client(project="acs-dp05-03-analysis")
# query the full ACS census tract table:
query = """
SELECT *
FROM `bigquery-public-data.census_bureau_acs.censustract_2020_5yr`
"""

# run query
query_job = client.query(query)

acs_df = query_job.result().to_dataframe(create_bqstorage_client=False)

print("Loaded ACS dataframe with shape:", acs_df.shape)
acs_df.head()

Loaded ACS dataframe with shape: (85395, 245)


,geo_id,aggregate_travel_time_to_work,amerindian_including_hispanic,amerindian_pop,armed_forces,asian_including_hispanic,asian_male_45_54,asian_male_55_64,asian_pop,associates_degree,...,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,walked_to_work,white_including_hispanic,white_male_45_54,white_male_55_64,white_pop,worked_at_home,workers_16_and_over
0,1001020200,18595.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,72.0,...,147.0,32.0,0.0,0.0,647.0,89.0,32.0,641.0,10.0,700.0
1,1001020300,47570.0,0.0,0.0,21.0,44.0,0.0,0.0,44.0,243.0,...,113.0,0.0,0.0,34.0,2363.0,143.0,232.0,2363.0,32.0,1959.0
2,1001020400,37660.0,10.0,10.0,10.0,17.0,0.0,0.0,17.0,257.0,...,105.0,23.0,25.0,28.0,3097.0,121.0,258.0,3085.0,45.0,1583.0
3,1001020503,28605.0,0.0,0.0,96.0,18.0,0.0,0.0,18.0,252.0,...,0.0,0.0,0.0,22.0,2397.0,218.0,105.0,2397.0,121.0,1606.0
4,1001020600,23255.0,0.0,0.0,40.0,11.0,0.0,0.0,11.0,145.0,...,134.0,43.0,22.0,6.0,2667.0,193.0,176.0,2551.0,80.0,1212.0


In [ ]:
# list of DP03 strings
dp03_cols = [
    'median_income', 'income_per_capita', 'poverty', 'pop_in_labor_force', 'employed_pop', 
    'unemployed_pop', 'not_in_labor_force', 'commuters_16_over', 'aggregate_travel_time_to_work', 
    'commute_less_10_mins', 'commute_5_9_mins', 'commute_10_14_mins', 'commute_15_19_mins', 
    'commute_20_24_mins', 'commute_25_29_mins', 'commute_30_34_mins', 'commute_35_39_mins', 
    'commute_40_44_mins', 'commute_45_59_mins', 'commute_60_more_mins', 'worked_at_home', 
    'walked_to_work', 'no_car', 'one_car', 'two_cars', 'three_cars', 'four_more_cars', 'gini_index'
]
dp03_df = acs_df[["geo_id"] + dp03_cols].copy()
dp03_df.head()


,geo_id,median_income,income_per_capita,poverty,pop_in_labor_force,employed_pop,unemployed_pop,not_in_labor_force,commuters_16_over,aggregate_travel_time_to_work,...,commute_45_59_mins,commute_60_more_mins,worked_at_home,walked_to_work,no_car,one_car,two_cars,three_cars,four_more_cars,gini_index
0,1001020200,49144.0,22772.0,257.0,729.0,688.0,29.0,721.0,690.0,18595.0,...,58.0,16.0,10.0,0.0,26.0,226.0,199.0,56.0,35.0,0.3746
1,1001020300,62423.0,25291.0,533.0,2015.0,1941.0,53.0,915.0,1927.0,47570.0,...,184.0,126.0,32.0,34.0,6.0,443.0,391.0,371.0,104.0,0.3604
2,1001020400,64310.0,47056.0,281.0,1668.0,1619.0,39.0,1391.0,1538.0,37660.0,...,56.0,109.0,45.0,28.0,68.0,590.0,661.0,298.0,15.0,0.5588
3,1001020503,81165.0,40522.0,319.0,1670.0,1547.0,27.0,1134.0,1485.0,28605.0,...,0.0,0.0,121.0,22.0,0.0,615.0,510.0,104.0,41.0,0.4089
4,1001020600,43030.0,24783.0,831.0,1263.0,1189.0,34.0,1548.0,1132.0,23255.0,...,37.0,47.0,80.0,6.0,0.0,461.0,446.0,307.0,99.0,0.4760


**CLEAN DATA**

In [ ]:
# create high burden rent target
acs_df['high_rent_burden'] = acs_df['percent_income_spent_on_rent'] >= 30
acs_df['high_rent_burden'].head()

0     True
1    False
2    False
3    False
4    False
Name: high_rent_burden, dtype: bool

In [ ]:
# Dim and shape
acs_df.shape
acs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85395 entries, 0 to 85394
Columns: 246 entries, geo_id to high_rent_burden
dtypes: bool(1), float64(244), object(1)
memory usage: 159.7+ MB


In [ ]:
# handlingsentinel codes
sentinel_values = [-666666666, -777777777, -888888888, -999999999]
acs_df.replace(sentinel_values, pd.NA, inplace=True)

In [ ]:
# visualize by county
acs_df["county_fips"] = acs_df["geo_id"].str[:5]

In [ ]:
# specify RI census tracts and scale numeric features
ri_df = dp03_df[dp03_df['geo_id'].str.startswith('1400000US44')].copy()
# install sklearn
%pip install scikit-learn
import sklearn
from sklearn.preprocessing import StandardScaler
numeric_cols = ri_df.select_dtypes(include=[np.number]).columns
scaler = StandardScaler()
ri_df_scaled = ri_df.copy()
ri_df_scaled[numeric_cols] = scaler.fit_transform(ri_df[numeric_cols])
ri_df_scaled.head()

Note: you may need to restart the kernel to use updated packages.


ValueError: Found array with 0 sample(s) (shape=(0, 28)) while a minimum of 1 is required by StandardScaler.